In [35]:
from openai import OpenAI
import json
import requests
import os
import pandas as pd
import numpy as np
import faiss

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=api_key,
)

data_paths = [
    "/workspaces/generative-ai-for-beginners/test/data/First.txt",
    "/workspaces/generative-ai-for-beginners/test/data/Second.txt",
    "/workspaces/generative-ai-for-beginners/test/data/Third.txt", 
] 

df = pd.DataFrame(columns=['path', 'text'])

for path in data_paths:
    try:
        with open(path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        
        new_row = pd.DataFrame({'path': [path], 'text': [file_content]})
        df = pd.concat([df, new_row], ignore_index=True)
    except FileNotFoundError:
        print(f"Файл не знайдено: {path}")

df.head()

def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

flattened_df = splitted_df.explode('chunks')

flattened_df.head()



embed_model_name = "text-embedding-3-small" 
def create_embeddings(text):
    if isinstance(text, pd.Series):
        text = text.iloc[0]
    
    if not isinstance(text, list):
        text = [str(text)]
    else:
        text = [str(item) for item in text]
    

    response = client.embeddings.create(
        input=text,
        model=embed_model_name 
    )
    
    return response.data[0].embedding


embeddings = create_embeddings(flattened_df['chunks'][0])
print(embeddings)


[-0.004604504909366369, 0.012229069136083126, 0.00420451769605279, 0.010957791469991207, 0.007745490409433842, -0.0020479969680309296, -0.005233942065387964, 0.052389033138751984, -0.07173725217580795, -0.015391758643090725, 0.059582602232694626, -0.013680185191333294, -0.00435955123975873, 0.024569761008024216, 0.0227589663118124, 0.007193570025265217, -0.013481741771101952, -0.049387577921152115, 0.01170195359736681, 0.006418400444090366, 0.02310624159872532, -0.02607048861682415, 0.05586178973317146, 0.02592165768146515, -0.0029906025156378746, 0.010374863632023335, 0.0304858535528183, -0.0034014421980828047, 6.506576028186828e-05, -0.0020138893742114305, 0.03475238382816315, -0.020675312727689743, -0.01776067540049553, -0.040457628667354584, -0.054869573563337326, 0.021853569895029068, -0.002576662227511406, 0.013233687728643417, 0.018467629328370094, 0.050627849996089935, 0.017338983714580536, -0.04886666312813759, -0.03214781731367111, 0.03678642585873604, -0.03078351728618145, -

In [36]:
import pickle
from pathlib import Path


def save_embeddings(df, folder="embeddings", filename="flattened_df.pkl"):
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    file_path = os.path.join(folder, filename)
    
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"DataFrame успішно збережено в {file_path}")

def load_embeddings(folder="embeddings", filename="flattened_df.pkl"):
    file_path = os.path.join(folder, filename)
    
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            df = pickle.load(f)
        
        print(f"DataFrame успішно завантажено з {file_path}")
        return df
    else:
        print(f"Файл {file_path} не знайдено")
        return None

def get_or_create_embeddings(df, chunk_column, embedding_function, folder="embeddings", filename="flattened_df.pkl"):
    loaded_df = load_embeddings(folder, filename)
    
    if loaded_df is not None:
        return loaded_df
    
    print("Створення нових ембедінгів...")
    
    embeddings = []
    for chunk in df[chunk_column]:
        embeddings.append(embedding_function(chunk))
    
    df['embeddings'] = embeddings
    
    save_embeddings(df, folder, filename)
    
    return df



flattened_df = get_or_create_embeddings(
    splitted_df.explode('chunks'), 
    'chunks', 
    create_embeddings
)

flattened_df.head()



Файл embeddings/flattened_df.pkl не знайдено
Створення нових ембедінгів...
DataFrame успішно збережено в embeddings/flattened_df.pkl


,path,text,chunks,embeddings
0,/workspaces/generative-ai-for-beginners/test/d...,Introduction to RoboticsMechanicsand ControlTh...,Introduction to RoboticsMechanicsand ControlTh...,"[-0.004604504909366369, 0.012229069136083126, ..."
0,/workspaces/generative-ai-for-beginners/test/d...,Introduction to RoboticsMechanicsand ControlTh...,ESM: David W. RiccardiExecutive Managing Edito...,"[0.042672693729400635, 0.022454578429460526, -..."
0,/workspaces/generative-ai-for-beginners/test/d...,Introduction to RoboticsMechanicsand ControlTh...,Lisa McDowellSenior Marketing Manager: Holly S...,"[0.03221004456281662, -0.019610971212387085, -..."
0,/workspaces/generative-ai-for-beginners/test/d...,Introduction to RoboticsMechanicsand ControlTh...,you purchased this book within the United Stat...,"[-0.000317789294058457, 0.01155538484454155, -..."
0,/workspaces/generative-ai-for-beginners/test/d...,Introduction to RoboticsMechanicsand ControlTh...,and publisher of this book have used their bes...,"[0.019555112347006798, 0.016201334074139595, 0..."


In [38]:
embeddings_list = flattened_df['embeddings'].to_list()
embeddings_array = np.array(embeddings_list).astype('float32')

vector_dimension = len(embeddings_list[0])

index = faiss.IndexFlatL2(vector_dimension)

index.add(embeddings_array)

print(f"Загальна кількість векторів в індексі: {index.ntotal}, розмірність векторів: {vector_dimension}")

question = "Що таке механічний маніпулятор?"

query_vector = create_embeddings(question)

query_vector_array = np.array([query_vector]).astype('float32')

k = 5
distances, indices = index.search(query_vector_array, k)

for i in range(min(3, len(indices[0]))):
    idx = indices[0][i]
    print(f"Фрагмент {i+1}:")
    print(flattened_df['chunks'].iloc[idx])
    print(f"Шлях: {flattened_df['path'].iloc[idx]}")
    print(f"Відстань: {distances[0][i]}")
    print("-" * 50)

Загальна кількість векторів в індексі: 1107, розмірність векторів: 1536
Фрагмент 1:
the motion of manipulators.2We use joint actuators as the generic term for devices that power a manipulator—for example,electric motors, hydraulic and pneumatic actuators, and muscles. 10Chapter 1Introduction T3( FIG URE 1.10:Therelationshipbetween the torques applied by the actuators andthe resulting
Шлях: /workspaces/generative-ai-for-beginners/test/data/First.txt
Відстань: 1.0944336652755737
--------------------------------------------------
Фрагмент 2:
Mechanisms and their Classification . .. 11 References ............................................... 18 2 Manipulator Kinematic Model (Manja Kircanski) . . . . . . . . . . . . . . . .. 19 2.1 Introduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .. 19 2.2 Definitions
Шлях: /workspaces/generative-ai-for-beginners/test/data/Second.txt
Відстань: 1.1101644039154053
-----------------------------------------------

In [41]:
deployment = "gpt-4o-mini"

def chatbot_with_rag(user_input):
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')

    k = 5
    distances, indices = index.search(query_vector_array, k)
    history = [flattened_df['chunks'].iloc[idx] for idx in indices[0]]

    context = "\n\n".join(history)

    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with AI questions."},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}\n\nProvide a brief but complete answer based on the context. Answer in Ukrainian."}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        temperature=0,
        max_tokens=400,
    )

    return response.choices[0].message.content


def chatbot_without_rag(user_input):
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. Provide brief but complete answers. Answer in Ukrainian."},
        {"role": "user", "content": f"Question: {user_input}"}
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        temperature=0,
        max_tokens=400,
    )

    return response.choices[0].message.content



from IPython.display import display, Markdown, HTML

def compare_responses(user_input, save_to_file=False, filename="rag_comparison.md"):
    query_vector = create_embeddings(user_input)

    query_vector_array = np.array([query_vector]).astype('float32')
    k = 5
    distances, indices = index.search(query_vector_array, k)
    
    rag_response = chatbot_with_rag(user_input)
    no_rag_response = chatbot_without_rag(user_input)
    
    markdown_text = f"""
# Порівняння відповідей

## 📝 Запит: {user_input}

## 🔍 Відповіді моделей

### Без використання RAG

{no_rag_response}

### З використанням RAG

{rag_response}

## 📚 Знайдені фрагменти тексту

"""
    
    for i, idx in enumerate(indices[0]):
        chunk_content = flattened_df['chunks'].iloc[idx]
        path = flattened_df['path'].iloc[idx]
        dist = float(distances[0][i])
        
        markdown_text += f"""
### Фрагмент {i+1} (відстань: {dist:.4f})

**Шлях**: {path}

{chunk_content}

"""
    
    display(Markdown(markdown_text))
    
    mathjax_script = """
    <script type="text/javascript">
        MathJax = {
            tex: {
                inlineMath: [['$', '$']]
            }
        };
    </script>
    <script type="text/javascript" id="MathJax-script" async
        src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
    </script>
    """
    display(HTML(mathjax_script))
    
    if save_to_file:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_text)
        print(f"Результати збережено у файл: {filename}")

compare_responses("Що таке автономна навігація?")


# Порівняння відповідей

## 📝 Запит: Що таке автономна навігація?

## 🔍 Відповіді моделей

### Без використання RAG

Автономна навігація — це процес визначення місцезнаходження та прокладання маршруту без участі людини, за допомогою різних технологій, таких як GPS, лазерні сканери, камери та інші сенсори. Ця технологія використовується в безпілотних транспортних засобах, роботах та інших системах, що потребують самостійного переміщення в просторі. Автономна навігація дозволяє системам адаптуватися до змін у навколишньому середовищі та приймати рішення в реальному часі.

### З використанням RAG

Автономна навігація — це процес, за допомогою якого роботи здатні самостійно приймати рішення та виконувати завдання, використовуючи зворотний зв'язок від датчиків для корекції своїх рухів і підтримки бажаної траєкторії. Це дозволяє роботам адаптуватися до змін у середовищі та виконувати завдання без необхідності постійного втручання людини.

## 📚 Знайдені фрагменти тексту


### Фрагмент 1 (відстань: 1.4787)

**Шлях**: /workspaces/generative-ai-for-beginners/test/data/Third.txt

are robots which are used to do advanced tasks and are programmed to make decisions on their own. When a robot is designed the most important thing to be kept in mind is that What the function is to be performed and what are the limitations of the robot. Each robot has a basic level of complexity and


### Фрагмент 2 (відстань: 1.4818)

**Шлях**: /workspaces/generative-ai-for-beginners/test/data/First.txt

butdoes not in itself constitute a solution. A primary concern of a position controlsystemis to compensate automatically for errors in knowledge of the parametersof a system and to suppress disturbances that tend to perturb the system from thedesired trajectory. To accomplish this, position and velocity


### Фрагмент 3 (відстань: 1.5029)

**Шлях**: /workspaces/generative-ai-for-beginners/test/data/First.txt

FIG U RE 1.13:In ordertocause the manipulator to follow the desired trajectory, aposition-control system must be implemented.Such a system uses feedback fromjoint sensors to keep the manipulator on course. Fig. 1.13.) In Chapter 9, we wifi consider control algorithms whose synthesis is basedon linear


### Фрагмент 4 (відстань: 1.5207)

**Шлях**: /workspaces/generative-ai-for-beginners/test/data/First.txt

robot itself. Acommon argument raised in their favor is that an off-line programming system wifinot cause production equipment (i.e., the robot) to be tied up when it needs to bereprogrammed; hence, automated factories can stay in production mode a greaterpercentage of the time. (See Fig. 1.16.)They also


### Фрагмент 5 (відстань: 1.5223)

**Шлях**: /workspaces/generative-ai-for-beginners/test/data/First.txt

fixed automation by being"flexible," which means programmable. Not only are the movements of manipulatorsprogrammable, but, through the use of sensors and communications with otherfactory automation, manipulators can adapt to variations as the task proceeds. (SeeFig. 1.15.)In typical robot systems, there

